# Intro 
Supervised Learning

Date Range: Jan 2021 to June 2021

Linear regression
- Target    = hospital admission rate
- Feature   = date, vaccination, case_new, case_active, case_unvax, case_fvax

KNN regression
- Target    = hospital admission rate
- Feature   = date, vaccination, case_new, case_active, case_unvax, case_fvax

Other Techniques:
Linear Reg, KNN, decision tree, random forest, SVM, Neural Network

# Prerequisite

In [2]:
import random
import time
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# TensorFlow
import tensorflow as tf
from tensorflow.keras import layers, models

# Loading DataSet from Github

In [3]:
# URL should be https://raw.githubusercontent.com/username/repository/branch/filename.csv so no download needed

# Cases, Age, Location, more: https://github.com/MoH-Malaysia/covid19-public/tree/main/epidemic/{specific_file}
# Vaccination: https://github.com/CITF-Malaysia/citf-public/tree/main/vaccination/{specific_file}

url_cases = "https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/cases_malaysia.csv"
url_hosp = "https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/hospital.csv"
url_vax = "https://raw.githubusercontent.com/CITF-Malaysia/citf-public/main/vaccination/vax_malaysia.csv"


In [ ]:
# Linear Regression
